In [ ]:
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

In [ ]:
from firedrake import *
import matplotlib.pyplot as plt

import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Ex.1 - Stokes problem - patch test

\begin{equation*}
\begin{cases}
- \Delta \boldsymbol{u} + \nabla  p  = \boldsymbol{0} & {\rm in} \ \Omega=(0,3)\times(0,1), \\
{\rm div}\,\boldsymbol{u} = 0 & {\rm in} \ \Omega, \\
(\nabla\boldsymbol{u}-pI)\boldsymbol{n} = \boldsymbol{0} & {\rm on} \ \Gamma_1\cup\Gamma_2,\\
\boldsymbol{u} = \boldsymbol{0} & {\rm on} \ \Gamma_3, \\
\boldsymbol{u} = \boldsymbol{g}_\text{D} & {\rm on} \ \Gamma_4.
\end{cases}
\end{equation*}

with $\boldsymbol{g}_\text{D} = 1\boldsymbol{i}$.

In [ ]:
from firedrake import RectangleMesh

# Build the mesh
n = 10
mesh = RectangleMesh(3*n, n, 3, 1)

fig, ax = plt.subplots()
triplot(mesh, axes=ax)
ax.legend()

In [ ]:
# Function spaces

# P1-P0
V = VectorFunctionSpace(mesh, 'P', 1)
Q = FunctionSpace(mesh, 'DP', 0) # NB: P0 are DISCONTINUOUS elements (DP)

# P1-P1
# V = VectorFunctionSpace(mesh, 'P', 1)
# Q = FunctionSpace(mesh, 'P', 1)

# P1b-P1
# The enrichment of the velocity space has to be done at the finite element level
V1_el = FiniteElement('CG', mesh.ufl_cell(), 1)
B_el = FiniteElement('Bubble', mesh.ufl_cell(), mesh.topological_dimension() + 1)
V_el = VectorElement(NodalEnrichedElement(V1_el, B_el))
V = FunctionSpace(mesh, V_el)
Q = FunctionSpace(mesh, 'P', 1)

# P2-P1
# V = VectorFunctionSpace(mesh, 'P', 2)
# Q = FunctionSpace(mesh, 'P', 1)

W = MixedFunctionSpace([V, Q])
print('Ndofs - velocity :',V.dim(),', pressure :',Q.dim(),', total :',W.dim())

# Finite element functions
u, p = TrialFunctions(W)
v, q = TestFunctions(W)

In [ ]:
# Boundary conditions (strong)
bc3 = DirichletBC(W.sub(0), Constant((0., 0.)), 3)
bc4 = DirichletBC(W.sub(0), Constant((1., 0.)), 4)
bcs = (bc3, bc4)

# Variational formulation
a = inner(grad(u), grad(v)) * dx - div(v) * p * dx + q * div(u) * dx
L = inner(Constant((0.0,0.0)), v) * dx
  # Dummy rhs (=0) to ensure that the solve recognize a==L as a linear problem

# Solution (NB: do not use the same name u,v,p,q of the trial/test functions)
wh = Function(W)
solve(a == L, wh, bcs=bcs)
uh, ph = wh.subfunctions

In [ ]:
# Variational formulation (penalty method) - we will see later on in the course
eps = 1.e-30
a = inner(grad(u), grad(v)) * dx - div(v) * p * dx + q * div(u) * dx \
    + 1./eps*inner(u,v)*ds(3) + 1./eps*inner(u,v)*ds(4)
L = 1./eps*inner(Constant((1.,0.)),v) * ds(4)

# Solution (NB: do not use the same name u,v,p,q of the trial/test functions)
wh = Function(W)
solve(a == L, wh, bcs=bcs)
uh, ph = wh.subfunctions

In [ ]:
fig, ax = plt.subplots()
col = tripcolor(ph, axes=ax)
plt.colorbar(col)
plt.title('pressure')
fig, ax = plt.subplots()
col = quiver(uh, axes=ax)
plt.colorbar(col)
plt.title('velocity')

In [ ]:
# Error computation
x = SpatialCoordinate(mesh)
u_ex = as_vector([x[1],0.])
grad_u_ex = as_tensor([[0.,1.],[0.,0.]])
p_ex = Constant(0.)
errL2u = sqrt(assemble( inner(uh-u_ex,uh-u_ex) * dx ))
errH10u = sqrt(assemble( inner(grad(uh)-grad_u_ex,grad(uh)-grad_u_ex) * dx ))
errL2p = sqrt(assemble( inner(ph-p_ex,ph-p_ex) * dx ))
print('Errors - L2-u:', errL2u, ', H10-u:', errH10u, ', L2-p:', errL2p)

## Ex.2 - Stokes problem - fully Dirichlet

In [ ]:
# Build the mesh
n = 10
mesh = UnitSquareMesh(n, n)

fig, ax = plt.subplots()
triplot(mesh, axes=ax)
ax.legend()

In [ ]:
# Function spaces
V = VectorFunctionSpace(mesh, 'P', 2)
Q = FunctionSpace(mesh, 'P', 1)
W = MixedFunctionSpace([V, Q])

# Exact solution and source term.
# In principle they are generic expressions, but in Firedrake they need to be
# defined on a specific mesh (see below for a more general definition).
x = SpatialCoordinate(mesh)
u_ex = as_vector((
    -cos(x[0]) * sin(x[1]),
    sin(x[0]) * cos(x[1])))
p_ex = -0.25 * (cos(2*x[0]) + cos(2*x[1])) + 0.25*sin(2.0)
# The last term in p_ex makes sure that the mean is zero, consistent with the
# null space of the B operator in the fully Dirichlet case.
f = as_vector((
    -2 * cos(x[0]) * sin(x[1]) + 0.5 * sin(2 * x[0]),
    2 * sin(x[0]) * cos(x[1]) + 0.5 * sin(2 * x[1])))

# Boundary conditions
bc = DirichletBC(W.sub(0), u_ex, (1,2,3,4))
bcs = (bc)

# Variational formulation
u, p = TrialFunctions(W)
v, q = TestFunctions(W)

a = inner(grad(u), grad(v)) * dx - div(v) * p * dx + q * div(u) * dx
L = dot(f, v) * dx

# Solution - default solver (iterative)
w = Function(W)
solve(a == L, w, bcs=bcs)
uh, ph = w.split()

# # Solution - direct solver
# w = Function(W)
# solve(a == L, w, bcs=bcs, solver_parameters={'ksp_type':'preonly', 'pc_type':'lu'})
# uh, ph = w.split()

# # Solution - iterative solver including pressure null space

# # Handle the pressure indetermination by specifying the nullspace
# # see https://www.firedrakeproject.org/solving-interface.html#singular-operators-in-mixed-spaces
# # We provide a basis for the nullspace of pressure (constant functions).
# # For velocity, we pass the whole space, signifying that we neglect it.
# nullspace = MixedVectorSpaceBasis(
#     W, [W.sub(0), VectorSpaceBasis(constant=True)]
# )

# w = Function(W)
# solve(a == L, w, bcs=bcs, nullspace=nullspace, solver_parameters={'ksp_type':'gmres', 'pc_type':'ilu'})
# uh, ph = w.split()

In [ ]:
fig, ax = plt.subplots()
q = tripcolor(ph, axes=ax)
plt.colorbar(q)
fig, ax = plt.subplots()
q = quiver(uh, axes=ax)
plt.colorbar(q)

### Convergence analysis

In [ ]:
# We collect all the solution steps in a function, so that the convergence test
# boils down to multiple calls to this function.

def solve_stokes(n, degreeU, degreeP, gD_fun, f_fun):
    # Mesh definition
    mesh = UnitSquareMesh(n, n, 'crossed')

    V = VectorFunctionSpace(mesh, 'P', degreeU)
    if degreeP==0:
        Q = FunctionSpace(mesh, 'DP', 0) #P0 element is discontinuous
    else:
        Q = FunctionSpace(mesh, 'P', degreeP)

    W = MixedFunctionSpace([V, Q])

    # The data generic expression are interpolated on the current mesh.
    x = SpatialCoordinate(mesh)
    gD = gD_fun(x)
    f = f_fun(x)

    bc = DirichletBC(W.sub(0), gD, 'on_boundary')

    # Handle the pressure indetermination by specifying the nullspace
    # see https://www.firedrakeproject.org/solving-interface.html#singular-operators-in-mixed-spaces
    # According to the documentation, we provide a basis for the nullspace of pressure
    # For velocity, we pass the whole space, signifying that "we don't care about" it
    nullspace = MixedVectorSpaceBasis(
        W, [W.sub(0), VectorSpaceBasis(constant=True)]
    )

    # Variational formulation
    u, p = TrialFunctions(W)
    v, q = TestFunctions(W)

    a = inner(grad(u), grad(v)) * dx - div(v) * p * dx + q * div(u) * dx
    L = dot(f, v) * dx

    # Solution
    w = Function(W)
    solve(a == L, w, bcs=bc, nullspace=nullspace)
    u, p = w.split()

    return u, p, mesh

In [ ]:
n_vec = np.array((10, 20, 40))
err_u_L2 = np.zeros(n_vec.shape[0])
err_u_H1 = np.zeros(n_vec.shape[0])
err_p_L2 = np.zeros(n_vec.shape[0])
err_p_H1 = np.zeros(n_vec.shape[0])

# Exact solution and source term as general functions:
# to avoid mesh dependence, we define them as lambda functions.
u_ex_fun = lambda x: as_vector((
    -cos(x[0]) * sin(x[1]),
    sin(x[0]) * cos(x[1])))
p_ex_fun = lambda x: -0.25 * (cos(2*x[0]) + cos(2*x[1])) + 0.25*sin(2.0)
f_fun = lambda x: as_vector((
    -2 * cos(x[0]) * sin(x[1]) + 0.5 * sin(2 * x[0]),
    2 * sin(x[0]) * cos(x[1]) + 0.5 * sin(2 * x[1])))

for ii in range(n_vec.shape[0]):
    n = n_vec[ii]
    uh, ph, mesh = solve_stokes(n, 2, 1, u_ex_fun, f_fun)

    # Interpolate lambda functions on current mesh
    x = SpatialCoordinate(mesh)
    u_ex = u_ex_fun(x)
    p_ex = p_ex_fun(x)

    mean_ph = assemble(ph*dx)
    print('mean pressure = ', mean_ph)
    print('mean exact pressure = ', assemble(p_ex*dx))
    # Notice that the average of the discrete pressure is non-neglible despite
    # we consider its null space in the linear solver.
    # This spoils pressure's convergence rate.

    err_u_L2[ii] = errornorm(u_ex, uh, 'L2')
    err_u_H1[ii] = errornorm(u_ex, uh, 'H1')
    # err_p_L2[ii] = errornorm(p_ex, ph, 'L2')
    err_p_L2[ii] = sqrt(assemble((ph-mean_ph - p_ex)*(ph-mean_ph - p_ex)*dx))
      # correction to restore the convergence rate

    print('n = ', n_vec[ii])
    print('L2 error, velocity = ', err_u_L2[ii])
    print('H1 error, velocity = ', err_u_H1[ii])
    print('L2 error, pressure = ', err_p_L2[ii])

In [ ]:
h_vec = list(1./el for el in n_vec)
plt.loglog(h_vec, err_u_L2, marker='o', label='u L2')
plt.loglog(h_vec, err_u_H1, marker='o', label='u H1')
plt.loglog(h_vec, err_p_L2, marker='+', label='p L2')
plt.loglog(h_vec, err_p_H1, marker='+', label='p H1')
plt.loglog(h_vec, h_vec, linestyle='--', label='h')
plt.loglog(h_vec, list(h**2 for h in h_vec), linestyle='--', label='h^2')
plt.loglog(h_vec, list(h**3 for h in h_vec), linestyle='--', label='h^3')

plt.legend()

### (additional) Nullspace investigation

In [ ]:
# Small problem to investigate eigenvalues
n=5
mesh = UnitSquareMesh(n, n, 'crossed')
V = VectorFunctionSpace(mesh, 'P', 2)
Q = FunctionSpace(mesh, 'P', 1)
W = MixedFunctionSpace([V, Q])
x = SpatialCoordinate(mesh)
gD = u_ex_fun(x)
f = f_fun(x)
bc = DirichletBC(W.sub(0), gD, 'on_boundary')
u, p = TrialFunctions(W)
v, q = TestFunctions(W)
a = inner(grad(u), grad(v)) * dx - div(v) * p * dx + q * div(u) * dx

# Monolithic matrix
A = assemble(a, bcs=(bc)).M.values

fig, ax = plt.subplots()
plt.spy(A)

w, vv = np.linalg.eig(A)
true_nullspace = vv[:, abs(w)<1e-14]

print('Dimension of the computed nullspace = ', true_nullspace.shape[1])
# print('a nullspace function: ', true_nullspace[:,0])

Qdim = Q.dim() #dimension of pressure space
print('small eigenvalues = ', w[abs(w)<1e-14])
#print('eigenvector = ', true_nullspace[-Qdim:,0])
# see https://www.firedrakeproject.org/firedrake.html#firedrake.function.Function

# Plot eigenmode
ps = Function(Q, val=true_nullspace[-Qdim:,0])
fig, ax = plt.subplots()
q = tripcolor(ps, axes=ax)


# Ex.3 - Stokes problem - Block algebraic formulation

In [ ]:
# Build domain mesh - y_min=-1
n = 32
mesh = RectangleMesh(2*n, 2*n, 2., 1., originY=-1.)

fig, ax = plt.subplots()
triplot(mesh, axes=ax)
ax.legend()

In [ ]:
from time import perf_counter

# Function spaces
V = VectorFunctionSpace(mesh, 'P', 2)
Q = FunctionSpace(mesh, 'P', 1)
W = MixedFunctionSpace([V, Q])

# Boundary conditions
x = SpatialCoordinate(mesh)
gD = (1.-x[1]**2, 0.)
bc_in = DirichletBC(W.sub(0), gD, 1)
bc_wall = DirichletBC(W.sub(0), Constant((0.,0.)), (3, 4))
bcs = (bc_in, bc_wall)

# Handle the pressure indetermination by specifying the nullspace
# see https://www.firedrakeproject.org/solving-interface.html#singular-operators-in-mixed-spaces
# Following the documentation, we provide a basis for the nullspace of pressure
# For velocity, we pass the whole space, signifying that "we don't care about" it
nullspace = MixedVectorSpaceBasis(
    W, [W.sub(0), VectorSpaceBasis(constant=True)]
)

# Variational formulation
u, p = TrialFunctions(W)
v, q = TestFunctions(W)
f = Constant((0.,0.))

a = 1.e-2*inner(grad(u), grad(v)) * dx - div(v) * p * dx + q * div(u) * dx
L = dot(f, v) * dx

# Solution
w = Function(W)

vpb = LinearVariationalProblem(a, L, w, bcs)

parameters = {'ksp_type': 'gmres', 'pc_type': 'bjacobi', 'sub_pc_type': 'ilu',
              'ksp_rtol': 1.e-5, 'ksp_max_it': 10000}
solver =  LinearVariationalSolver(vpb, solver_parameters=parameters)
t0 = perf_counter()
solver.solve()
print('elapsed time = ', perf_counter() - t0, 's    -    # iter = ', solver.snes.ksp.getIterationNumber())

In [ ]:
uh, ph = w.split()
fig, ax = plt.subplots()
q = tripcolor(ph, axes=ax)
plt.colorbar(q)
fig, ax = plt.subplots()
q = quiver(uh, axes=ax)
plt.colorbar(q)